In [1]:
import pandas as pd
import numpy as np
from mido import Message, MidiFile, MidiTrack, MetaMessage

In [119]:
df = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv')

In [120]:
df.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,07/09/2020,7,9,2020,74,2,Afghanistan,AF,AFG,38041757.0,Asia,1.048847
1,06/09/2020,6,9,2020,20,0,Afghanistan,AF,AFG,38041757.0,Asia,0.854324
2,05/09/2020,5,9,2020,16,0,Afghanistan,AF,AFG,38041757.0,Asia,1.077763
3,04/09/2020,4,9,2020,45,1,Afghanistan,AF,AFG,38041757.0,Asia,1.135594
4,03/09/2020,3,9,2020,38,3,Afghanistan,AF,AFG,38041757.0,Asia,1.272286


In [121]:
df['dateRep'] = pd.to_datetime(df['dateRep'], format = '%d/%m/%Y')

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41210 entries, 0 to 41209
Data columns (total 12 columns):
 #   Column                                                      Non-Null Count  Dtype         
---  ------                                                      --------------  -----         
 0   dateRep                                                     41210 non-null  datetime64[ns]
 1   day                                                         41210 non-null  int64         
 2   month                                                       41210 non-null  int64         
 3   year                                                        41210 non-null  int64         
 4   cases                                                       41210 non-null  int64         
 5   deaths                                                      41210 non-null  int64         
 6   countriesAndTerritories                                     41210 non-null  object        
 7   geoId                 

In [179]:
centroids_df = pd.read_csv('./data/country_centroids_az8.csv')
continent_centroids_df = pd.read_csv(
    '~/Dropbox (BBC)/Visual Journalism/Data/2020/vjdata.26866.global.covid.interactive/\
derived/bbc_world_v2_4326_centroid_region.csv'
)

In [180]:
continent_centroids_df

,english,iso3,latLong,bbc-region-code
0,Serbia,SRB,"44.4141272993523, 20.7902071905549",region_eu
1,British Indian Ocean Territory,IOT,"-7.19195760979793, 72.2988070103603",region_af
2,Guyana,GUY,"4.82541876028894, -58.9800866249185",region_am_lac
3,Honduras,HND,"14.9163778987637, -86.6252566181024",region_am_lac
4,Iraq,IRQ,"33.2145658654154, 43.7452613942025",region_me
...,...,...,...,...
247,Guadeloupe,GLP,"16.3117359483936, -61.5296102785394",region_am_lac
248,Martinique,MTQ,"14.7508911390534, -61.0176171003129",region_am_lac
249,West Bank,PSE,"32.1183424354869, 35.2449534672761",region_me
250,Somalia,SOM,"6.10230769124075, 45.8505338259364",region_af


In [181]:
continent_centroids_df['bbc-region-code'].unique()

array(['region_eu', 'region_af', 'region_am_lac', 'region_me',
       'region_as', 'region_am_na', nan, 'region_oc', 'region_an'],
      dtype=object)

In [87]:
for code in df['countryterritoryCode'].unique(): 
    if not code in centroids_df['adm0_a3'].values:
        print(code)

BES
nan
GIB
XKX
MSF
PSE
SSD
CNG1925
ESH


In [123]:
df = pd.merge(df, 
         centroids_df[['adm0_a3', 'Longitude', 'Latitude', 'region_wb']], 
         left_on='countryterritoryCode', 
         right_on = 'adm0_a3')

In [124]:
df.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000,adm0_a3,Longitude,Latitude
0,2020-09-07,7,9,2020,74,2,Afghanistan,AF,AFG,38041757.0,Asia,1.048847,AFG,66.004734,33.835231
1,2020-09-06,6,9,2020,20,0,Afghanistan,AF,AFG,38041757.0,Asia,0.854324,AFG,66.004734,33.835231
2,2020-09-05,5,9,2020,16,0,Afghanistan,AF,AFG,38041757.0,Asia,1.077763,AFG,66.004734,33.835231
3,2020-09-04,4,9,2020,45,1,Afghanistan,AF,AFG,38041757.0,Asia,1.135594,AFG,66.004734,33.835231
4,2020-09-03,3,9,2020,38,3,Afghanistan,AF,AFG,38041757.0,Asia,1.272286,AFG,66.004734,33.835231


In [125]:
df = df.sort_values(by = ['countryterritoryCode', 'dateRep']).reset_index(drop=True)

In [127]:
df['cumulative_deaths'] = df.groupby(['countryterritoryCode'])['deaths'].cumsum()
df['cumulative_cases'] = df.groupby(['countryterritoryCode'])['cases'].cumsum()

In [128]:
df

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000,adm0_a3,Longitude,Latitude,cumulative_deaths,cumulative_cases
0,2020-03-13,13,3,2020,2,0,Aruba,AW,ABW,106310.0,America,NaN,ABW,-69.982677,12.520880,0,2
1,2020-03-20,20,3,2020,2,0,Aruba,AW,ABW,106310.0,America,NaN,ABW,-69.982677,12.520880,0,4
2,2020-03-24,24,3,2020,8,0,Aruba,AW,ABW,106310.0,America,NaN,ABW,-69.982677,12.520880,0,12
3,2020-03-25,25,3,2020,5,0,Aruba,AW,ABW,106310.0,America,NaN,ABW,-69.982677,12.520880,0,17
4,2020-03-26,26,3,2020,2,0,Aruba,AW,ABW,106310.0,America,NaN,ABW,-69.982677,12.520880,0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39737,2020-09-03,3,9,2020,79,3,Zimbabwe,ZW,ZWE,14645473.0,Africa,6.793908,ZWE,29.851441,-19.004204,206,6638
39738,2020-09-04,4,9,2020,40,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,6.370569,ZWE,29.851441,-19.004204,206,6678
39739,2020-09-05,5,9,2020,159,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,6.978266,ZWE,29.851441,-19.004204,206,6837
39740,2020-09-06,6,9,2020,0,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,6.445678,ZWE,29.851441,-19.004204,206,6837


In [129]:
df['cases_rolling_average'] = df.groupby('countryterritoryCode').apply(lambda group: group["cases"].rolling(7).mean()).reset_index(drop=True)

In [133]:
df['cases_rolling_average'] = df.groupby('countryterritoryCode').rolling(7)['cases'].mean().reset_index(drop=True)
df['deaths_rolling_average'] = df.groupby('countryterritoryCode').rolling(7)['deaths'].mean().reset_index(drop=True)

In [134]:
df.tail(60)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000,adm0_a3,Longitude,Latitude,cumulative_deaths,cumulative_cases,cases_rolling_average,deaths_rolling_average
39682,2020-07-10,10,7,2020,41,3,Zimbabwe,ZW,ZWE,14645473.0,Africa,2.560518,ZWE,29.851441,-19.004204,12,926,44.142857,0.714286
39683,2020-07-11,11,7,2020,16,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,2.601486,ZWE,29.851441,-19.004204,13,942,45.285714,0.857143
39684,2020-07-12,12,7,2020,40,5,Zimbabwe,ZW,ZWE,14645473.0,Africa,2.833640,ZWE,29.851441,-19.004204,18,982,40.571429,1.428571
39685,2020-07-13,13,7,2020,3,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,2.854124,ZWE,29.851441,-19.004204,18,985,38.428571,1.428571
39686,2020-07-14,14,7,2020,49,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,3.140902,ZWE,29.851441,-19.004204,19,1034,42.857143,1.428571
39687,2020-07-15,15,7,2020,0,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,3.024825,ZWE,29.851441,-19.004204,19,1034,35.285714,1.428571
39688,2020-07-16,16,7,2020,55,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,3.304775,ZWE,29.851441,-19.004204,20,1089,29.142857,1.571429
39689,2020-07-17,17,7,2020,273,3,Zimbabwe,ZW,ZWE,14645473.0,Africa,5.086896,ZWE,29.851441,-19.004204,23,1362,62.285714,1.571429
39690,2020-07-18,18,7,2020,58,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,5.428299,ZWE,29.851441,-19.004204,24,1420,68.285714,1.571429
39691,2020-07-19,19,7,2020,58,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,5.325878,ZWE,29.851441,-19.004204,25,1478,70.857143,1.000000


In [136]:
df.loc[df['countryterritoryCode'] == 'GBR', ['cases', 'deaths', 'cases_rolling_average', 'deaths_rolling_average']].tail(60)



,cases,deaths,cases_rolling_average,deaths_rolling_average
13646,693,31,590.142857,36.142857
13647,715,34,606.285714,34.000000
13648,565,17,604.285714,31.857143
13649,442,9,610.142857,30.428571
13650,361,10,582.428571,30.285714
13651,726,44,585.571429,28.857143
13652,685,26,598.142857,24.428571
13653,772,24,609.428571,23.428571
13654,704,26,607.857143,22.285714
13655,569,9,608.428571,21.142857


In [137]:
top_20_worst_hit_countries = df.groupby(
    'countryterritoryCode'
)['deaths'].sum().sort_values(ascending = False)[:20].index

In [138]:
top_50_worst_hit_countries = df.groupby(
    'countryterritoryCode'
)['deaths'].sum().sort_values(ascending = False)[:50].index

In [139]:
top_30_worst_hit_countries = df.groupby(
    'countryterritoryCode'
)['deaths'].sum().sort_values(ascending = False)[:30].index

In [140]:
diff = df['dateRep'].max() - df['dateRep'].min()

In [141]:
diff.days

251

In [142]:
df.loc[df['countryterritoryCode'].isin(top_30_worst_hit_countries), 'continentExp'].unique()

array(['America', 'Europe', 'Asia', 'Africa'], dtype=object)

## Version one below: chord progression from maj7 to diminished, daily pulse, velocity changes with cases


In [39]:
max_cases_value = df.loc[df['countryterritoryCode'].isin(top_30_worst_hit_countries), 'cumulative_cases'].max()
# max_death_value = df.loc[df['countryterritoryCode'].isin(top_30_worst_hit_countries), 'cumulative_deaths'].max()
# threshold_width = max_death_value / 5

# Different keys, not so harmonious:
# root_notes = {
#     'Asia': 55, 
#     'Europe': 60, 
#     'America': 65, 
#     'Africa': 70
# }

root_notes = {
    'Asia': 55, 
    'Europe': 55, 
    'America': 55, 
    'Africa': 55
}

instrumentation = {
    'Asia': 68, # oboe 
    'Europe': 74, # flute
    'America': 70, # bassoon
    'Africa': 12 # marimba
}

channel_numbers = {continent: channel_no for channel_no, continent in enumerate(instrumentation.keys())}

duration = 4

class Pulse(): 
    def __init__(self, chord, duration, velocity, root = 55): 
        self.chord = chord
        self.root = root
        self.notes = self.define_notes()
        self.duration = duration
        self.velocity = int(velocity)
        
    def define_notes(self): 
        if self.chord == 'maj7': 
            return [self.root, self.root + 4, self.root + 7, self.root + 11]
        elif self.chord == 'min/maj7': 
            return [self.root, self.root + 3, self.root + 7, self.root + 11]
        elif self.chord == 'dim/maj7': 
            return [self.root, self.root + 3, self.root + 6, self.root + 11]
        elif self.chord == 'ø7': 
            return [self.root, self.root + 3, self.root + 6, self.root + 10]
        elif self.chord == 'dim': 
            return [self.root, self.root + 3, self.root + 6, self.root + 9]
        
    def add_to_track(self, track): 
        for note in self.notes: 
            message = Message('note_on', note = note, velocity = self.velocity, time = 0)
            track.append(message)
        for idx, note in enumerate(self.notes): 
            if idx == 0: 
                time = 120 * self.duration
            else: 
                time = 0
            message = Message('note_on', note = note, velocity = 0, time = time)
            track.append(message)

mid = MidiFile()
            
for country_code in top_30_worst_hit_countries: 
    country_df = df.loc[df['countryterritoryCode'] == country_code]
    lon = country_df['Longitude'].values[0]
    continent = country_df['continentExp'].values[0]
    channel_no = channel_numbers[continent]
    cases_data = country_df['cumulative_cases']
    deaths_data = country_df['cumulative_deaths']
    if len(deaths_data) < diff.days: 
        missing_days = diff.days - len(deaths_data)
        cases_data = np.pad(cases_data, [missing_days, 0], mode = 'constant')
        deaths_data = np.pad(deaths_data, [missing_days, 0], mode = 'constant')
#     X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
#     X_scaled = X_std * (max - min) + min
    lon_std = (lon - -150) / (150 - -150)
    pan = int(lon_std * 127)
    
    track = MidiTrack()
    mid.tracks.append(track)
    track.append(MetaMessage('track_name', name = country_code, time = 0))
    instrument = instrumentation[continent]
    track.append(Message('program_change', channel=channel_no, program=instrument, time=0))
    track.append(Message('control_change', channel=channel_no, control=10, value=pan, time=0))
    
#     max_cases_value = cases_data.max()
    max_death_value = deaths_data.max()
    threshold_width = max_death_value / 5
    
    for cases, deaths in zip(cases_data, deaths_data): 
        if cases > 0: 
            velocity = np.log10(cases) / np.log10(max_cases_value) * 127
        else: 
            velocity = 0
        root = root_notes[continent]
        if deaths <= threshold_width: 
            pulse = Pulse(chord = 'maj7', duration = duration, velocity = velocity, root = root)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 2: 
            pulse = Pulse(chord = 'min/maj7', duration = duration, velocity = velocity, root = root)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 3: 
            pulse = Pulse(chord = 'dim/maj7', duration = duration, velocity = velocity, root = root)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 4: 
            pulse = Pulse(chord = 'ø7', duration = duration, velocity = velocity, root = root)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 5: 
            pulse = Pulse(chord = 'dim', duration = duration, velocity = velocity, root = root)
            pulse.add_to_track(track)

    mid.save('./output/test_021.mid')

## Version two: chord stays on maj7 but ascends octaves, weekly pulse, velocity stays same but volume changes with cases


In [55]:
max_cases_value = df.loc[df['countryterritoryCode'].isin(top_30_worst_hit_countries), 'cumulative_cases'].max()
# max_death_value = df.loc[df['countryterritoryCode'].isin(top_30_worst_hit_countries), 'cumulative_deaths'].max()
# threshold_width = max_death_value / 5

instrumentation = {
    'Asia': 68, # oboe 
    'Europe': 74, # flute
    'America': 70, # bassoon
    'Africa': 12 # marimba
}

channel_numbers = {continent: channel_no for channel_no, continent in enumerate(instrumentation.keys())}

duration = 12

class Pulse(): 
    def __init__(self, chord, duration, velocity, root = 55, volume_changes = []): 
        self.chord = chord
        self.root = root
        self.notes = self.define_notes()
        self.duration = duration
        self.velocity = int(velocity)
        self.volume_changes = volume_changes
        
    def define_notes(self): 
        if self.chord == 'maj7': 
            return [self.root, self.root + 4, self.root + 7, self.root + 11]
        elif self.chord == 'min/maj7': 
            return [self.root, self.root + 3, self.root + 7, self.root + 11]
        elif self.chord == 'dim/maj7': 
            return [self.root, self.root + 3, self.root + 6, self.root + 11]
        elif self.chord == 'ø7': 
            return [self.root, self.root + 3, self.root + 6, self.root + 10]
        elif self.chord == 'dim': 
            return [self.root, self.root + 3, self.root + 6, self.root + 9]
        
    def add_to_track(self, track, channel): 
        for note in self.notes: 
            message = Message('note_on', channel = channel, note = note, velocity = self.velocity, time = 0)
            track.append(message)
        elapsed_time = 0
        for volume in self.volume_changes:
            delay = int(120 * self.duration / 7)
            elapsed_time += delay
            track.append(Message('control_change', channel=channel_no, control=7, value=volume, time=delay))
        for idx, note in enumerate(self.notes): 
            if idx == 0: 
                time = 120 * self.duration - elapsed_time
            else: 
                time = 0
            message = Message('note_on', channel = channel, note = note, velocity = 0, time = time)
            track.append(message)

mid = MidiFile()
            
for country_code in top_30_worst_hit_countries: 
    country_df = df.loc[df['countryterritoryCode'] == country_code]
    lon = country_df['Longitude'].values[0]
    continent = country_df['continentExp'].values[0]
    channel_no = channel_numbers[continent]
    cases_data = country_df['cumulative_cases']
    deaths_data = country_df['cumulative_deaths']
    if len(deaths_data) < diff.days: 
        missing_days = diff.days - len(deaths_data)
        cases_data = pd.Series(np.pad(cases_data, [missing_days, 0], mode = 'constant'))
        deaths_data = np.pad(deaths_data, [missing_days, 0], mode = 'constant')
#     X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
#     X_scaled = X_std * (max - min) + min
    lon_std = (lon - -150) / (150 - -150)
    pan = int(lon_std * 127)
    
    track = MidiTrack()
    mid.tracks.append(track)
    track.append(MetaMessage('track_name', name = country_code, time = 0))
    instrument = instrumentation[continent]
    track.append(Message('program_change', channel=channel_no, program=instrument, time=0))
    track.append(Message('control_change', channel=channel_no, control=10, value=pan, time=0))
    track.append(Message('control_change', channel=channel_no, control=7, value=0, time=0))
    
#     max_cases_value = cases_data.max()
    max_death_value = deaths_data.max()
    threshold_width = max_death_value / 5
    
    day_number = 0
    velocity = 100
    
    def convert_cases_to_volume(cases): 
        if cases > 0: 
            return int(np.log10(cases) / np.log10(max_cases_value) * 127)
        return 0
    for cases, deaths in zip(cases_data, deaths_data): 
        volume_changes = list(cases_data[day_number:day_number+7].map(convert_cases_to_volume))
#         print(volume_changes)
#         track.append(Message('control_change', channel=channel_no, control=7, value=volume, time=(day_number - 1)%7))
        root = 44
        interval = 7
        if day_number % 7 == 0: 
            if deaths <= threshold_width: 
                root = root + interval
                pulse = Pulse(chord = 'maj7', duration = duration, velocity = velocity, root = root, volume_changes = volume_changes)
                pulse.add_to_track(track, channel=channel_no)
            elif deaths <= threshold_width * 2: 
                root = root + interval * 2
                pulse = Pulse(chord = 'maj7', duration = duration, velocity = velocity, root = root, volume_changes = volume_changes)
                pulse.add_to_track(track, channel=channel_no)
            elif deaths <= threshold_width * 3: 
                root = root + interval * 3
                pulse = Pulse(chord = 'maj7', duration = duration, velocity = velocity, root = root, volume_changes = volume_changes)
                pulse.add_to_track(track, channel=channel_no)
            elif deaths <= threshold_width * 4: 
                root = root + interval * 4
                pulse = Pulse(chord = 'maj7', duration = duration, velocity = velocity, root = root, volume_changes = volume_changes)
                pulse.add_to_track(track, channel=channel_no)
            elif deaths <= threshold_width * 5:  
                root = root + interval * 5
                pulse = Pulse(chord = 'maj7', duration = duration, velocity = velocity, root = root, volume_changes = volume_changes)
                pulse.add_to_track(track, channel=channel_no)
        day_number += 1

    mid.save('./output/v2_test_004.mid')

## Version three below: chord progression from maj7 to diminished, daily pulse, velocity changes with rolling average of daily cases


In [150]:
max_cases_value = df.loc[
    df['countryterritoryCode'].isin(top_30_worst_hit_countries), 
    'cases_rolling_average'
].max()
# max_death_value = df.loc[df['countryterritoryCode'].isin(top_30_worst_hit_countries), 'cumulative_deaths'].max()
# threshold_width = max_death_value / 5

# Different keys, not so harmonious:
# root_notes = {
#     'Asia': 55, 
#     'Europe': 60, 
#     'America': 65, 
#     'Africa': 70
# }

root_notes = {
    'Asia': 55, 
    'Europe': 55, 
    'America': 55, 
    'Africa': 55
}

instrumentation = {
    'Asia': 68, # oboe 
    'Europe': 74, # flute
    'America': 70, # bassoon
    'Africa': 12 # marimba
}

channel_numbers = {continent: channel_no for channel_no, continent in enumerate(instrumentation.keys())}

duration = 4

class Pulse(): 
    def __init__(self, chord, duration, velocity, root = 55): 
        self.chord = chord
        self.root = root
        self.notes = self.define_notes()
        self.duration = duration
        self.velocity = int(velocity)
        
    def define_notes(self): 
        if self.chord == 'maj7': 
            return [self.root, self.root + 4, self.root + 7, self.root + 11]
        elif self.chord == 'min/maj7': 
            return [self.root, self.root + 3, self.root + 7, self.root + 11]
        elif self.chord == 'dim/maj7': 
            return [self.root, self.root + 3, self.root + 6, self.root + 11]
        elif self.chord == 'ø7': 
            return [self.root, self.root + 3, self.root + 6, self.root + 10]
        elif self.chord == 'dim': 
            return [self.root, self.root + 3, self.root + 6, self.root + 9]
        
    def add_to_track(self, track): 
        for note in self.notes: 
            message = Message('note_on', note = note, velocity = self.velocity, time = 0)
            track.append(message)
        for idx, note in enumerate(self.notes): 
            if idx == 0: 
                time = 120 * self.duration
            else: 
                time = 0
            message = Message('note_on', note = note, velocity = 0, time = time)
            track.append(message)

mid = MidiFile()
            
for country_code in top_30_worst_hit_countries: 
    country_df = df.loc[df['countryterritoryCode'] == country_code]
    lon = country_df['Longitude'].values[0]
    continent = country_df['continentExp'].values[0]
    channel_no = channel_numbers[continent]
    cases_data = country_df['cases_rolling_average'].apply(lambda x: 0 if np.isnan(x) else int(x))
    deaths_data = country_df['cumulative_deaths']
    if len(deaths_data) < diff.days: 
        missing_days = diff.days - len(deaths_data)
        cases_data = np.pad(cases_data, [missing_days, 0], mode = 'constant')
        deaths_data = np.pad(deaths_data, [missing_days, 0], mode = 'constant')
#     X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
#     X_scaled = X_std * (max - min) + min
    lon_std = (lon - -150) / (150 - -150)
    pan = int(lon_std * 127)
    
    track = MidiTrack()
    mid.tracks.append(track)
    track.append(MetaMessage('track_name', name = country_code, time = 0))
    instrument = instrumentation[continent]
    track.append(Message('program_change', channel=channel_no, program=instrument, time=0))
    track.append(Message('control_change', channel=channel_no, control=10, value=pan, time=0))
    
#     max_cases_value = cases_data.max()
    max_death_value = deaths_data.max()
    threshold_width = max_death_value / 5
    
    for cases, deaths in zip(cases_data, deaths_data): 
        if cases > 0: 
#             velocity = np.log10(cases) / np.log10(max_cases_value) * 127
            velocity = cases / max_cases_value * 127
        else: 
            velocity = 0
        root = root_notes[continent]
        if deaths <= threshold_width: 
            pulse = Pulse(chord = 'maj7', duration = duration, velocity = velocity, root = root)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 2: 
            pulse = Pulse(chord = 'min/maj7', duration = duration, velocity = velocity, root = root)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 3: 
            pulse = Pulse(chord = 'dim/maj7', duration = duration, velocity = velocity, root = root)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 4: 
            pulse = Pulse(chord = 'ø7', duration = duration, velocity = velocity, root = root)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 5: 
            pulse = Pulse(chord = 'dim', duration = duration, velocity = velocity, root = root)
            pulse.add_to_track(track)

    mid.save('./output/v3_test_002.mid')

## Version four below: group by continent, chord progression from maj7 to diminished depending on cumulative deaths, daily pulse, divisi arpeggios, velocity changes with rolling average of daily cases


In [183]:
continent_centroids_df

,english,iso3,latLong,bbc-region-code
0,Serbia,SRB,"44.4141272993523, 20.7902071905549",region_eu
1,British Indian Ocean Territory,IOT,"-7.19195760979793, 72.2988070103603",region_af
2,Guyana,GUY,"4.82541876028894, -58.9800866249185",region_am_lac
3,Honduras,HND,"14.9163778987637, -86.6252566181024",region_am_lac
4,Iraq,IRQ,"33.2145658654154, 43.7452613942025",region_me
...,...,...,...,...
247,Guadeloupe,GLP,"16.3117359483936, -61.5296102785394",region_am_lac
248,Martinique,MTQ,"14.7508911390534, -61.0176171003129",region_am_lac
249,West Bank,PSE,"32.1183424354869, 35.2449534672761",region_me
250,Somalia,SOM,"6.10230769124075, 45.8505338259364",region_af


In [191]:
continent_df = pd.merge(df, 
         continent_centroids_df[['iso3', 'bbc-region-code']], 
         left_on = 'countryterritoryCode', 
         right_on = 'iso3').groupby(['dateRep', 'bbc-region-code'])['cases', 'deaths'].sum().reset_index()

/Users/lowthe01/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


In [194]:
def rename_continent(bbc_region): 
    switcher = {
        "region_as": "Asia",
        "region_af": "Africa",
        "region_am_na": "North America",
        "region_am_lac": "Latin America & Caribbean",
        "region_me": "Middle East",
        "region_eu": "Europe",
        "region_oc": "Oceania"
    }
    return switcher[bbc_region]
continent_df['continent_name'] = continent_df['bbc-region-code'].map(rename_continent)

In [196]:
continent_df = continent_df.sort_values(by = ['continent_name', 'dateRep']).reset_index(drop=True)

In [197]:
continent_df['cases_rolling_average'] = continent_df.groupby(
    'continent_name'
).rolling(7)['cases'].mean().reset_index(drop=True)
continent_df['deaths_rolling_average'] = continent_df.groupby(
    'continent_name'
).rolling(7)['deaths'].mean().reset_index(drop=True)

In [203]:
continent_df['cumulative_deaths'] = continent_df.groupby(['continent_name'])['deaths'].cumsum()

In [200]:
continent_df['continent_name'].unique()

array(['Africa', 'Asia', 'Europe', 'Latin America & Caribbean',
       'Middle East', 'North America', 'Oceania'], dtype=object)

In [301]:
weekly_continent_df = continent_df.groupby(
    ['continent_name', continent_df['dateRep'].dt.strftime('%W')]
)[['cases', 'deaths', 'cases_rolling_average']].agg({'cases': 'sum', 
                                                    'deaths': 'sum', 
                                                    'cases_rolling_average': 'mean'}).reset_index()

In [302]:
weekly_continent_df

,continent_name,dateRep,cases,deaths,cases_rolling_average
0,Africa,00,0,0,NaN
1,Africa,01,0,0,0.000000
2,Africa,02,0,0,0.000000
3,Africa,03,0,0,0.000000
4,Africa,04,0,0,0.000000
...,...,...,...,...,...
261,Oceania,33,2089,109,341.000000
262,Oceania,34,1810,120,269.714286
263,Oceania,35,1286,154,210.714286
264,Oceania,36,85,5,177.000000


In [303]:
weekly_continent_df = weekly_continent_df.sort_values(by = ['continent_name', 'dateRep']).reset_index(drop=True)

In [289]:
# weekly_continent_df['cases_rolling_average'] = weekly_continent_df.groupby(
#     'continent_name'
# ).rolling(7)['cases'].mean().reset_index(drop=True)
# weekly_continent_df['deaths_rolling_average'] = weekly_continent_df.groupby(
#     'continent_name'
# ).rolling(7)['deaths'].mean().reset_index(drop=True)

In [304]:
weekly_continent_df['cumulative_deaths'] = weekly_continent_df.groupby(['continent_name'])['deaths'].cumsum()

In [305]:
weekly_continent_df = weekly_continent_df.loc[weekly_continent_df['dateRep'] != '52']

In [306]:
weekly_continent_df.head(60)

,continent_name,dateRep,cases,deaths,cases_rolling_average,cumulative_deaths
0,Africa,00,0,0,NaN,0
1,Africa,01,0,0,0.000000,0
2,Africa,02,0,0,0.000000,0
3,Africa,03,0,0,0.000000,0
4,Africa,04,0,0,0.000000,0
5,Africa,05,0,0,0.000000,0
6,Africa,06,0,0,0.000000,0
7,Africa,07,0,0,0.000000,0
8,Africa,08,4,0,0.204082,0
9,Africa,09,25,0,2.265306,0


In [309]:
max_cases_value = weekly_continent_df['cases_rolling_average'].max()
# max_death_value = df.loc[df['countryterritoryCode'].isin(top_30_worst_hit_countries), 'cumulative_deaths'].max()
# threshold_width = max_death_value / 5

# previously was 48, 60, 72, 48, 60, 72:
root_notes = {
    'Asia': 48,
    'Middle East': 60,
    'Africa': 72,
    'Europe': 48, 
    'Latin America & Caribbean': 60, 
    'North America': 72
}

pan_values = {
    'Asia': 120,
    'Middle East': 100,
    'Africa': 80,
    'Europe': 63, 
    'Latin America & Caribbean': 40, 
    'North America': 10
}

# instrumentation = {
#     'Asia': 68,
#     'Middle East': 74,
#     'Africa': 70,
#     'Europe': 12, 
#     'Latin America & Caribbean': 68, 
#     'North America': 74
# #     'Asia': 68, # oboe 
# #     'Europe': 74, # flute
# #     'America': 70, # bassoon
# #     'Africa': 12 # marimba
# }

instrumentation = {
    'Asia': 42, # cello
    'Middle East': 73, # flute
    'Africa': 41, # viola
    'Europe': 71, # clarinet
    'Latin America & Caribbean': 40, # violin
    'North America': 40 # violin
}

channel_numbers = {continent: channel_no for channel_no, continent in enumerate(instrumentation.keys())}

duration = 4

class Pulse(): 
    def __init__(self, chord, duration, velocity, root = 55, shimmer = 'out'): 
        self.chord = chord
        self.root = root
        self.shimmer = shimmer
        self.notes = self.define_notes()
        self.duration = duration
        self.velocity = int(velocity)
        
    def define_notes(self): 
        if self.chord == 'maj7': 
            return [self.root, self.root + 4, self.root + 7, self.root + 11]
        elif self.chord == 'min/maj7': 
            return [self.root, self.root + 3, self.root + 7, self.root + 11]
        elif self.chord == 'dim/maj7': 
            return [self.root, self.root + 3, self.root + 6, self.root + 11]
        elif self.chord == 'ø7': 
            return [self.root, self.root + 3, self.root + 6, self.root + 10]
        elif self.chord == 'dim': 
            return [self.root, self.root + 3, self.root + 6, self.root + 9]
        
    def add_breath(self, track):
        max_volume = self.velocity
        min_volume = max_volume - 20
        for volume in range(min_volume, max_volume + 1): 
            if volume < 0: 
                volume = 0
            track.append(Message('control_change', control=7, value=volume, time = 40))
        for volume in range(min_volume, max_volume + 1)[::-1]: 
            if volume < 0: 
                volume = 0
            track.append(Message('control_change', control=7, value=volume, time = 40))
        
    def add_to_track(self, track): 
        starting_volume = self.velocity - 20
        if starting_volume < 0: 
            starting_volume = 0
        track.append(Message('control_change', control=7, value=starting_volume))
        
        if self.shimmer == 'out':
        
            message = Message('note_on', note = self.notes[1], velocity = 64, time = 0)
            track.append(message)
            message = Message('note_on', note = self.notes[2], velocity = 64, time = 0)
            track.append(message)
            
            self.add_breath(track)

            message = Message('note_on', note = self.notes[1], velocity = 0, time = 0)#120 * self.duration)
            track.append(message)
            message = Message('note_on', note = self.notes[2], velocity = 0, time = 0)
            track.append(message)

            message = Message('note_on', note = self.notes[0], velocity = 64, time = 0)
            track.append(message)
            message = Message('note_on', note = self.notes[3], velocity = 64, time = 0)
            track.append(message)
            
            self.add_breath(track)

            message = Message('note_on', note = self.notes[0], velocity = 0, time = 0)#120 * self.duration)
            track.append(message)
            message = Message('note_on', note = self.notes[3], velocity = 0, time = 0)
            track.append(message)
        
        else: 
        
            message = Message('note_on', note = self.notes[0], velocity = 64, time = 0)
            track.append(message)
            message = Message('note_on', note = self.notes[3], velocity = 64, time = 0)
            track.append(message)
            
            self.add_breath(track)

            message = Message('note_on', note = self.notes[0], velocity = 0, time = 0)#120 * self.duration)
            track.append(message)
            message = Message('note_on', note = self.notes[3], velocity = 0, time = 0)
            track.append(message)

            message = Message('note_on', note = self.notes[1], velocity = 64, time = 0)
            track.append(message)
            message = Message('note_on', note = self.notes[2], velocity = 64, time = 0)
            track.append(message)
            
            self.add_breath(track)

            message = Message('note_on', note = self.notes[1], velocity = 0, time = 0)#120 * self.duration)
            track.append(message)
            message = Message('note_on', note = self.notes[2], velocity = 0, time = 0)
            track.append(message)
            
        
#         for note in self.notes[1:3]: 
#             message = Message('note_on', note = note, velocity = self.velocity, time = 0)
#             track.append(message)
#         for idx, note in enumerate(self.notes[1:3]): 
#             if idx == 0: 
#                 time = 120 * self.duration
#             else: 
#                 time = 0
#             message = Message('note_on', note = note, velocity = 0, time = time)
#             track.append(message)
#         for idx, note in enumerate([self.notes[0], self.notes[3]]): 
#             if idx == 0: 
#                 time = 120 * self.duration
#             else: 
#                 time = 0
#             message = Message('note_on', note = note, velocity = 0, time = time)
#             track.append(message)

mid = MidiFile()
            
diff_weeks = int(weekly_continent_df['dateRep'].max()) - int(weekly_continent_df['dateRep'].min())
for continent in ['Africa', 'Asia', 'Europe', 'Latin America & Caribbean', 'Middle East', 'North America']: 
    if continent in ['Africa', 'Europe', 'Middle East']:
        shimmer = 'out'
    else: 
        shimmer = 'in'
    filtered_df = weekly_continent_df.loc[weekly_continent_df['continent_name'] == continent]
    channel_no = channel_numbers[continent]
    cases_data = filtered_df['cases_rolling_average'].apply(lambda x: 0 if np.isnan(x) else int(x))
    deaths_data = weekly_continent_df.groupby('dateRep')['cumulative_deaths'].sum().values
    if len(deaths_data) < diff_weeks: 
        missing_weeks = diff_weeks - len(deaths_data)
        cases_data = np.pad(cases_data, [missing_weeks, 0], mode = 'constant')
        deaths_data = np.pad(deaths_data, [missing_weeks, 0], mode = 'constant')
#     X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
#     X_scaled = X_std * (max - min) + min
    lon_std = (lon - -150) / (150 - -150)
    pan = pan_values[continent]
    
    track = MidiTrack()
    mid.tracks.append(track)
    track.append(MetaMessage('track_name', name = continent, time = 0))
    instrument = instrumentation[continent]
    track.append(Message('program_change', channel=channel_no, program=instrument, time=0))
    track.append(Message('control_change', channel=channel_no, control=10, value=pan, time=0))
    
#     max_cases_value = cases_data.max()
    max_death_value = deaths_data.max()
    threshold_width = max_death_value / 5
    
    for cases, deaths in zip(cases_data, deaths_data): 
        if cases > 0: 
#             velocity = np.log10(cases) / np.log10(max_cases_value) * 127
            velocity = cases / max_cases_value * 100 + 26
        else: 
            velocity = 0
        root = root_notes[continent]
        if deaths <= threshold_width: 
            pulse = Pulse(chord = 'maj7', duration = duration, velocity = velocity, root = root, shimmer = shimmer)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 2: 
            pulse = Pulse(chord = 'min/maj7', duration = duration, velocity = velocity, root = root, shimmer = shimmer)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 3: 
            pulse = Pulse(chord = 'dim/maj7', duration = duration, velocity = velocity, root = root, shimmer = shimmer)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 4: 
            pulse = Pulse(chord = 'ø7', duration = duration, velocity = velocity, root = root, shimmer = shimmer)
            pulse.add_to_track(track)
        elif deaths <= threshold_width * 5: 
            pulse = Pulse(chord = 'dim', duration = duration, velocity = velocity, root = root, shimmer = shimmer)
            pulse.add_to_track(track)

    mid.save('./output/v4_test_019.mid')
    
    
    
    

In [248]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)
track.append(MetaMessage('track_name', name = 'testing', time = 0))
track.append(Message('program_change', channel=1, program=40, time=0))
for volume in range(127):
    track.append(Message('control_change', control=7, value=volume))
    track.append(Message('note_on', note = 55, time = 0))
    track.append(Message('note_off', note = 55, time = 360))
mid.save('./output/velocity_values_test.mid')

In [284]:
for country_code in top_30_worst_hit_countries: 
    country_df = df.loc[df['countryterritoryCode'] == country_code]
    cases_data = country_df['cumulative_cases']
    deaths_data = country_df['cumulative_deaths']
    print(country_code, cases_data.max())

USA 1961185
GBR 287399
BRA 691758
ITA 235278
FRA 154188
ESP 241717
MEX 120102
BEL 59348
DEU 184543
IRN 173832
CAN 96233
IND 266598
NLD 47739
RUS 476658
PER 199696
TUR 171121
SWE 45133
CHN 84194
ECU 43378
CHL 138846
PAK 108317
IDN 32033
IRL 25207
CHE 30889
PRT 34885
ROU 20604
COL 40719
EGY 35444
POL 27160
ZAF 50879


Useful for debugging:

In [271]:
for msg in mid.tracks[0]:
    print(msg)

<meta message track_name name='Africa' time=0>
program_change channel=2 program=41 time=0
control_change channel=2 control=10 value=80 time=0
control_change channel=0 control=7 value=0 time=0
note_on channel=0 note=76 velocity=64 time=0
note_on channel=0 note=79 velocity=64 time=0
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change cha

control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
control_change channel=0 control=7 value=0 time=2
note_on channel=0 note=72 velocity=0 time=480
note_on channel=0 note=83 velocity=0 time=0
control_change channel=0 control=7 value=0 time=0
note_on channel=0 note=76 velocity=64 time=0
note_on channel=0 note=79 velocity=64 time=0
control_change chann

note_on channel=0 note=72 velocity=0 time=480
note_on channel=0 note=83 velocity=0 time=0
control_change channel=0 control=7 value=16 time=0
note_on channel=0 note=76 velocity=64 time=0
note_on channel=0 note=79 velocity=64 time=0
control_change channel=0 control=7 value=16 time=2
control_change channel=0 control=7 value=17 time=2
control_change channel=0 control=7 value=18 time=2
control_change channel=0 control=7 value=19 time=2
control_change channel=0 control=7 value=20 time=2
control_change channel=0 control=7 value=21 time=2
control_change channel=0 control=7 value=22 time=2
control_change channel=0 control=7 value=23 time=2
control_change channel=0 control=7 value=24 time=2
control_change channel=0 control=7 value=25 time=2
control_change channel=0 control=7 value=26 time=2
control_change channel=0 control=7 value=26 time=2
control_change channel=0 control=7 value=25 time=2
control_change channel=0 control=7 value=24 time=2
control_change channel=0 control=7 value=23 time=2
cont

control_change channel=0 control=7 value=19 time=2
control_change channel=0 control=7 value=18 time=2
note_on channel=0 note=75 velocity=0 time=480
note_on channel=0 note=79 velocity=0 time=0
note_on channel=0 note=72 velocity=64 time=0
note_on channel=0 note=83 velocity=64 time=0
control_change channel=0 control=7 value=18 time=2
control_change channel=0 control=7 value=19 time=2
control_change channel=0 control=7 value=20 time=2
control_change channel=0 control=7 value=21 time=2
control_change channel=0 control=7 value=22 time=2
control_change channel=0 control=7 value=23 time=2
control_change channel=0 control=7 value=24 time=2
control_change channel=0 control=7 value=25 time=2
control_change channel=0 control=7 value=26 time=2
control_change channel=0 control=7 value=27 time=2
control_change channel=0 control=7 value=28 time=2
control_change channel=0 control=7 value=28 time=2
control_change channel=0 control=7 value=27 time=2
control_change channel=0 control=7 value=26 time=2
cont

control_change channel=0 control=7 value=37 time=2
control_change channel=0 control=7 value=38 time=2
control_change channel=0 control=7 value=39 time=2
control_change channel=0 control=7 value=40 time=2
control_change channel=0 control=7 value=41 time=2
control_change channel=0 control=7 value=42 time=2
control_change channel=0 control=7 value=42 time=2
control_change channel=0 control=7 value=41 time=2
control_change channel=0 control=7 value=40 time=2
control_change channel=0 control=7 value=39 time=2
control_change channel=0 control=7 value=38 time=2
control_change channel=0 control=7 value=37 time=2
control_change channel=0 control=7 value=36 time=2
control_change channel=0 control=7 value=35 time=2
control_change channel=0 control=7 value=34 time=2
control_change channel=0 control=7 value=33 time=2
control_change channel=0 control=7 value=32 time=2
note_on channel=0 note=75 velocity=0 time=480
note_on channel=0 note=78 velocity=0 time=0
note_on channel=0 note=72 velocity=64 time=

control_change channel=0 control=7 value=36 time=2
control_change channel=0 control=7 value=35 time=2
control_change channel=0 control=7 value=34 time=2
control_change channel=0 control=7 value=33 time=2
control_change channel=0 control=7 value=32 time=2
control_change channel=0 control=7 value=31 time=2
control_change channel=0 control=7 value=30 time=2
control_change channel=0 control=7 value=29 time=2
control_change channel=0 control=7 value=28 time=2
control_change channel=0 control=7 value=27 time=2
control_change channel=0 control=7 value=26 time=2
note_on channel=0 note=72 velocity=0 time=480
note_on channel=0 note=81 velocity=0 time=0
control_change channel=0 control=7 value=25 time=0
note_on channel=0 note=75 velocity=64 time=0
note_on channel=0 note=78 velocity=64 time=0
control_change channel=0 control=7 value=25 time=2
control_change channel=0 control=7 value=26 time=2
control_change channel=0 control=7 value=27 time=2
control_change channel=0 control=7 value=28 time=2
cont